In [1]:
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


import matplotlib.pyplot as plt
import os

In [5]:
# 超參數設定
batch_size =  128 #訓練完128張照片就更新一次權重
num_classes = 10 #照片分類總共分為10類
epochs = 20 #全部的相片跑100次

#儲存model
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

#資料載入
(x_train, y_train),(x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 將目標值轉為 one hot 型式
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

#建立模型
model = Sequential()
model.add(Conv2D(64, (3,3), padding = 'same',
                input_shape = x_train.shape[1:]))

print(x_train.shape[1:])

model.add(Activation('relu'))
model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

print(model.summary())

#選擇優化器
opt = keras.optimizers.Adam()

#編輯模型
model.compile(loss = 'categorical_crossentropy',
             optimizer = opt,
             metrics=['accuracy'])
             
#資料歸一化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.

print('Using real-time data augmentation.')

#數據再造 : 對圖片做旋轉、平移、模糊化...等操作
datagen = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    vertical_flip = False)

#儲存 模型與權重
if not os.path.isdir(save_dir):
    os.makeidrs(save_dir)
    
model_path = os.path.join(save_dir, model_name)
checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)

#設定早停功能,避免資源浪費
earlystop = EarlyStopping(monitor = 'val_loss', patience = 5, verbose = 1)#為什麼要有verbose


#訓練模型
model_history = model.fit_generator(datagen.flow(x_train, y_train
                                                , batch_size = batch_size),
                                    epochs = epochs,
                                    validation_data = (x_test, y_test), #什麼意思?
                                    callbacks = [earlystop])
#入載模型
print("Loading trained model")
model = load_model(model_path)

#評估模型
scores = model.evaluate(x_test, y_test, verbose = 1)
print("Test Loss", scores[0])
print("Test accuracy", scores[1])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
(32, 32, 3)
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_18 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 30, 30, 128)       73856     
_________________________________________________________________
activation_19 (Activation)   (None, 30, 30, 128)       0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 15, 15, 128)       0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 15, 15, 128)       0         
___________________________________

Test Loss 0.8156274256706237
Test accuracy 0.7139
